In [20]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State

data = pd.read_csv('/Users/sebastianwozniczka/Documents/dashboard-1/data/shopping_trends_updated.csv')
df = pd.DataFrame(data)

# Mapowanie nazw stanów do skrótów do mapy
df['Location'] = df['Location'].map({
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
})

app = Dash()

# Layout aplikacji (wykresy i pasek do wybierania kategorii)
app.layout = html.Div([
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': category, 'value': category} for category in df['Category'].unique()],
        value='Clothing'
    ),
    dcc.Graph(id='us-map'),
    dcc.Graph(id='bar-chart'),
    dcc.Graph(id='line-chart'),
    dcc.Slider(
        id='days-slider',
        min=1,
        max=30,
        value=10,
        marks={i: str(i) for i in range(1, 31)},
        step=1
    ),
    html.Button('Toggle to Units', id='toggle-button', n_clicks=0),
    dcc.Graph(id='category-pie-chart'),
    dcc.Graph(id='subcategory-histogram'),
    dcc.Graph(id='payment-method-pie-chart'),
    dcc.Slider(
        id='payments-slider',
        min=1,
        max=50,
        value=10,
        marks={i: str(i) for i in range(1, 51)},
        step=1
    ),
    dcc.Graph(id='payment-method-line-chart')
])

# Mapa USA
@app.callback(
    Output('us-map', 'figure'),
    Input('category-dropdown', 'value')
)
def update_us_map(selected_category):
    filtered_df = df[df['Category'] == selected_category]
    grouped_df = filtered_df.groupby('Location')['Purchase Amount (USD)'].sum().reset_index()
    fig = px.choropleth(data_frame=grouped_df, locations='Location', locationmode='USA-states', color='Purchase Amount (USD)',
                        scope='usa', title=f'Sales of {selected_category} by State')
    return fig

# wykres slupkowy (domyslnie texas)
@app.callback(
    Output('bar-chart', 'figure'),
    Input('us-map', 'clickData')
)
def update_bar_chart(clickData):
    if clickData is None:
        state = 'TX'  
    else:
        state = clickData['points'][0]['location']
    
    filtered_df = df[df['Location'] == state]
    grouped_df = filtered_df.groupby('Category')['Purchase Amount (USD)'].sum().reset_index()
    bar_chart = px.bar(grouped_df, x='Category', y='Purchase Amount (USD)',
                       title=f'Sales by Category in {state}')
    return bar_chart

# wykres liniowy
@app.callback(
    Output('line-chart', 'figure'),
    [Input('category-dropdown', 'value'),
     Input('us-map', 'clickData'),
     Input('days-slider', 'value'),
     Input('toggle-button', 'n_clicks')]
)
def update_line_chart(selected_category, clickData, days, n_clicks):
    if clickData is None:
        state = 'TX'  
    else:
        state = clickData['points'][0]['location']
    
    filtered_df = df[(df['Category'] == selected_category) & (df['Location'] == state)]
    recent_sales = filtered_df.tail(days)
    recent_sales = recent_sales.reset_index()
    recent_sales['Days Ago'] = recent_sales.index + 1
    
    if n_clicks % 2 == 0:
        line_chart = px.line(recent_sales, x='Days Ago', y='Purchase Amount (USD)',
                             title=f'Last {days} Sales of {selected_category} in {state}')
    else:
        line_chart = px.line(recent_sales, x='Days Ago', y='Quantity Sold',
                             title=f'Last {days} Sales (Units) of {selected_category} in {state}')
    
    return line_chart

# wykres kołowy z rozkładem % sprzedaży per kategoria w stanie
@app.callback(
    Output('category-pie-chart', 'figure'),
    Input('us-map', 'clickData')
)
def update_category_pie_chart(clickData):
    if clickData is None:
        return px.pie(title='Select a state to see category distribution')
    
    state = clickData['points'][0]['location']
    filtered_df = df[df['Location'] == state]
    grouped_df = filtered_df.groupby('Category')['Purchase Amount (USD)'].sum().reset_index()
    total_amount = grouped_df['Purchase Amount (USD)'].sum()
    grouped_df['Percentage'] = (grouped_df['Purchase Amount (USD)'] / total_amount) * 100
    pie_chart = px.pie(grouped_df, names='Category', values='Percentage',
                       title=f'Sales Percentage by Category in {state}')
    return pie_chart

# histogram z podkategoriami danej kategorii
@app.callback(
    Output('subcategory-histogram', 'figure'),
    [Input('category-dropdown', 'value'),
     Input('us-map', 'clickData')]
)
def update_subcategory_histogram(selected_category, clickData):
    if clickData is None:
        state = 'TX'  
    else:
        state = clickData['points'][0]['location']
    
    filtered_df = df[(df['Category'] == selected_category) & (df['Location'] == state)]
    grouped_df = filtered_df.groupby('Item Purchased')['Purchase Amount (USD)'].sum().reset_index()
    histogram = px.histogram(grouped_df, x='Purchase Amount (USD)', y='Item Purchased', orientation='h',
                             title=f'Sales by Subcategory in {state}')
    return histogram

#wykres kolowy rodzaje platnosci
@app.callback(
    Output('payment-method-pie-chart', 'figure'),
    Input('category-dropdown', 'value')
)
def update_payment_method_pie_chart(selected_category):
    pie_chart = px.pie(df, names='Payment Method', values='Purchase Amount (USD)',
                       title='Sales Distribution by Payment Method')
    return pie_chart

# liniowy rodzaje platnosci (domyslnie credit card)
@app.callback(
    Output('payment-method-line-chart', 'figure'),
    [Input('payment-method-pie-chart', 'clickData'),
     Input('payments-slider', 'value')]
)
def update_payment_method_line_chart(clickData, payments):
    if clickData is None:
        payment_method = 'Credit Card'  
    else:
        payment_method = clickData['points'][0]['label']
    
    filtered_df = df[df['Payment Method'] == payment_method]
    recent_payments = filtered_df.tail(payments)
    recent_payments = recent_payments.reset_index()
    recent_payments['Days Ago'] = recent_payments.index + 1
    line_chart = px.line(recent_payments, x='Days Ago', y='Purchase Amount (USD)',
                         title=f'Last {payments} Payments by {payment_method}')
    return line_chart

if __name__ == '__main__':
    app.run_server(debug=True)